In [28]:
import pandas
from fppylec import main, forecast, optimization
from datetime import datetime, timedelta
import plotly.express as px
import logging
from tabulate import tabulate
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
%load_ext autoreload
%autoreload 2

layout = {
    'showlegend': False,
    'margin': {'b':10, 'l':20, 'r':50, 't':50},
    'font': {'size': 19},
    'xaxis': {'zerolinewidth': 2,  'zerolinecolor':'white'},
    'yaxis': {'zerolinewidth': 2, 'zerolinecolor':'white'},
    'template': 'plotly_dark',
}
px.defaults.color_discrete_sequence = px.colors.qualitative.T10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Inputs

In [26]:
# Filenames (correct folder)
batch = 0
nb_houses = [10, 30, 95]
inputfiles = [f'data/{x}_{batch}.pickle' for x in nb_houses]
outputfile = 'result.pickle'

# Simulation period
start= '2019-05-01 00:00:00'
start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
end = '2019-05-10 23:45:00'
end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
f_horizon = timedelta(days=2) - timedelta(minutes=15)
f_train_period = timedelta(days=14)

# Simulation variable (forecasting method)
f_method = forecast.ma
f_kwarg = {}
control_func = optimization.pool

# Running simulation

In [30]:
_start_time = datetime.now()
metrics = pandas.DataFrame()

for inputfile, nb_house in zip(inputfiles, nb_houses):
    print(f'Using {inputfile} out of {inputfiles}')
    df = pandas.read_pickle(inputfile)
    df = df.loc[:, ['vo_houses_kW', 'vo_pv_coef']]

    # Assert timing
    assert start in df.index
    assert end in df.index
    assert int(pandas.infer_freq(df.index)[:-1]) == 15

    # Assert overall consumption (20000kWh is a lot)
    assert df['vo_houses_kW'].sum() * 15/60 / nb_house < 20000, f'Total conso {df["vo_houses_kW"].sum()}'

    # Assert PV
    assert df['vo_pv_coef'].min() >= 0
    assert df['vo_pv_coef'].max() <= 1.01, f'Max PV coef = {df["vo_pv_coef"].max()}'
    
    ###########
    ### RUN ###
    ###########
    pv_capacity = 3 * nb_house  # kW
    battery_kWh = 6 * nb_house   # kWh
    battery_kW = 2 * nb_house  # kW
    result = main.main(df.asfreq('15T'), start, end, pv_capacity, battery_kWh, battery_kW,
                  f_method, f_kwarg, f_horizon, f_train_period, control_func)
    
    # Save
    result['index'] = nb_house
    result['filename'] = inputfile
    result['algo'] = f'{f_method.__name__}'
    result['batch'] = batch
    metrics = metrics.append(result, ignore_index=True)

print(tabulate(
    metrics[['index', 'MAPE_%', 'MAPE_9a8p_%', 'MASE',
             'scons_%', 'ssuff_%',
             'ME_kW', 'MAE_kW']],
    headers='keys', tablefmt='psql'))
print('')
print('------')
_time_elapsed = datetime.now() - _start_time
print('Total time elapsed (hh:mm:ss) {}'.format(_time_elapsed))
print('')

Using data/10_0.pickle out of ['data/10_0.pickle', 'data/30_0.pickle', 'data/95_0.pickle']
Time elapsed (hh:mm:ss.ms) 0:00:02.145610

Using data/30_0.pickle out of ['data/10_0.pickle', 'data/30_0.pickle', 'data/95_0.pickle']
Time elapsed (hh:mm:ss.ms) 0:00:02.143897

Using data/95_0.pickle out of ['data/10_0.pickle', 'data/30_0.pickle', 'data/95_0.pickle']
Time elapsed (hh:mm:ss.ms) 0:00:02.290902

+----+---------+----------+---------------+--------+-----------+-----------+---------+----------+
|    |   index |   MAPE_% |   MAPE_9a8p_% |   MASE |   scons_% |   ssuff_% |   ME_kW |   MAE_kW |
|----+---------+----------+---------------+--------+-----------+-----------+---------+----------|
|  0 |      10 |    34.72 |         25.66 |   2.24 |   75.0851 |   68.709  |   -0.07 |     1.49 |
|  1 |      30 |    22.94 |         17.46 |   2.29 |   77.8298 |   72.2838 |    0.26 |     2.96 |
|  2 |      95 |    20.56 |         13.52 |   3.03 |   77.1945 |   73.6328 |   -0.53 |     7.94 |
+----+----

# Visualization

In [ ]:
# plot metrics as a function of nb houses

In [ ]:
# Draw relation between value and quality